In [1]:
include("../src/Julia.jl")

parseData (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
gpath = "../../graphs/";

In [4]:
graphs = readdir("../../graphs");
graphs = graphs[ceil(Int64,length(graphs)*3/4)+1:end]

6-element Array{ByteString,1}:
 "rand_1000_iu1000"     
 "rand_1000_u10"        
 "rand_1000_u100"       
 "rand_1000_u1000"      
 "randdense_1000_iu1000"
 "randdense_1000_u1000" 

In [5]:
for cur in graphs
    path = gpath * cur * "/"
    
    try
        a = readFromFile(path * "graph.mtx");
        x = readFromFile(path * "x.vec");
        
        tree = []
        for ind in 1:5
            tr = readFromFile(path * "tree$(ind).mtx");
            
            u,v,w = findnz(tr)
            for i in 1:length(w)
                w[i] = a[u[i],v[i]]
            end
            tr = sparse(u,v,w)
            
            push!(tree, tr);
        end
        
        la = lap(a);
        x = x - mean(x)
        b = la * x; b = b - mean(b);
        
        numIts = 1000;
        @time for i in 1:5
            @time F = treeSolver(tree[i])
            @time F(b);

            # a run on standard Float64 data types
            @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
            norm(lap(a) * myx64 - b) / norm(b)
            insert!(dbg, 1, "$(numIts) iterations")
            insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
            writeToFile(path * "_log_julia_tree$(i)_default.txt", dbg)

            # now useing higer precision data types
            for hp in [64,128,256,512,1024]
                set_bigfloat_precision(hp)
                laHP = toHighPrecision(la; precision=hp)
                treeHP = toHighPrecision(tree[i]; precision=hp)
                xHP = toHighPrecision(x; precision=hp);
                bHP = toHighPrecision(b; precision=hp);
                FHP = treeSolver(treeHP)

                @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=false);
                insert!(dbg, 1, "$(numIts) iterations")
                insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
                writeToFile(path * "_log_julia_tree$(i)_$(hp).txt", dbg)
            end
        end
            
        println("finished running on ", cur, "\n")
    catch
        println("skipping ", cur, "\n")
    end
end

  0.582474 seconds (606.99 k allocations: 25.261 MB, 1.04% gc time)
  0.196522 seconds (110.95 k allocations: 4.243 MB)
  1.360057 seconds (18.22 M allocations: 462.149 MB, 5.36% gc time)
130.129120 seconds (617.14 M allocations: 15.489 GB, 23.68% gc time)
132.878067 seconds (616.93 M allocations: 17.005 GB, 23.98% gc time)
149.038840 seconds (616.93 M allocations: 20.055 GB, 23.45% gc time)
166.754190 seconds (616.93 M allocations: 26.154 GB, 24.45% gc time)
186.622002 seconds (616.93 M allocations: 38.351 GB, 27.28% gc time)
  0.001765 seconds (5.14 k allocations: 586.766 KB)
  0.000734 seconds (17.93 k allocations: 343.875 KB)
  1.023241 seconds (18.01 M allocations: 452.567 MB, 14.13% gc time)
134.793327 seconds (616.93 M allocations: 15.480 GB, 24.49% gc time)
144.730259 seconds (616.93 M allocations: 17.005 GB, 24.74% gc time)
154.875164 seconds (616.93 M allocations: 20.055 GB, 24.30% gc time)
161.211690 seconds (616.93 M allocations: 26.154 GB, 24.59% gc time)
187.786764 second